**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Mon Feb  8 08:44:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
 chatsettingsbackup.db.crypt1	    msgstore-2020-10-19.1.db.crypt12
'Colab Notebooks'		    msgstore.db.crypt12
'msgstore (1).db.crypt12'	   'My Drive'
 msgstore-2020-10-12.1.db.crypt12   pothole_detection
 msgstore-2020-10-13.1.db.crypt12   statusranking.db.crypt1
 msgstore-2020-10-14.1.db.crypt12   steel_surface_detection
 msgstore-2020-10-15.1.db.crypt12   stickers.db.crypt1
 msgstore-2020-10-17.1.db.crypt12   yolov3
 msgstore-2020-10-18.1.db.crypt12


**1) Clone, configure & compile Darknet**



In [ ]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


In [ ]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

[Errno 20] Not a directory: 'darknet'
/content/darknet


In [ ]:
# Compile
!make

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_opencv.cpp:1130:10: note: ...thi

**2) Configure yolov3.cfg file**

In [ ]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 2000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

**3) Create .names and .data files**

In [ ]:
!echo -e 'pothole' > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/pothole_detection' > data/obj.data

**4) Save yolov3_training.cfg and obj.names files in Google drive**

In [ ]:
!cp cfg/yolov3_training.cfg /mydrive/pothole_detection/yolov3_testing.cfg
!cp data/obj.names /mydrive/pothole_detection/classes.txt

**5) Create a folder and unzip image dataset**

In [ ]:
!mkdir data/obj
!unzip /mydrive/pothole_detection/images.zip -d data/obj

mkdir: cannot create directory ‘data/obj’: File exists
Archive:  /mydrive/pothole_detection/images.zip
replace data/obj/img-3_jpg.rf.4a9b39b2dee6163e6e919bd81fe5e12b.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/obj/img-3_jpg.rf.4a9b39b2dee6163e6e919bd81fe5e12b.txt  
  inflating: data/obj/img-4_jpg.rf.f1d66cc8bfcc312214832c952d2b45b0.jpg  
  inflating: data/obj/img-4_jpg.rf.f1d66cc8bfcc312214832c952d2b45b0.txt  
  inflating: data/obj/img-5_jpg.rf.3f3de0a94403d4da7300a15057f62d60.jpg  
  inflating: data/obj/img-5_jpg.rf.3f3de0a94403d4da7300a15057f62d60.txt  
  inflating: data/obj/img-6_jpg.rf.f6375830aa98fb073a356ad52ebe8741.jpg  
  inflating: data/obj/img-6_jpg.rf.f6375830aa98fb073a356ad52ebe8741.txt  
  inflating: data/obj/img-7_jpg.rf.077c45fc14bc4c1e14e1c6d2ceebdebd.jpg  
  inflating: data/obj/img-7_jpg.rf.077c45fc14bc4c1e14e1c6d2ceebdebd.txt  
  inflating: data/obj/img-8_jpg.rf.cf7b0bc58abebaef0bfcd5681703b598.jpg  
  inflating: data/obj/img-8_jpg.rf.cf7b0bc58abeba

**6) Create train.txt file**

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

**7) Download pre-trained weights for the convolutional layers file**

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-02-08 10:55:07--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M   128KB/s    in 17m 36s 

2021-02-08 11:12:43 (150 KB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



**8) Start training**

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 299000, rewritten_bbox = 0.003344 % 

 1926: 0.458769, 0.481535 avg loss, 0.001000 rate, 5.450291 seconds, 123264 images, 0.172472 hours left
Loaded: 0.000035 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.775270), count: 5, class_loss = 1.015821, iou_loss = 0.243708, total_loss = 1.259528 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.843955), count: 2, class_loss = 0.209933, iou_loss = 0.040284, total_loss = 0.250216 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.753039), count: 3, class_loss = 0.331423, iou_loss = 0.171187, total_loss = 0.502610 
 total_bbox = 299010, rewritten_bbox = 0.003344 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.791739), count: 5, class_loss = 0.581359, iou_loss = 0.115892, total_loss = 0.697252 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1